In [12]:
import numpy as np
import pandas as pd

In [16]:
# load the ratings CSV files
df1_ratings = pd.read_csv("https://drive.google.com/uc?export=view&id=1BAMVkLqMrXJDYNaLC973Jcd_r77WCgQn")
df2_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1Pc60nR9sxXfmPIsJRvfhqYYOO5zA3VsT&export=download")
df3_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1tkMrm_--FZWps1uXMwEF3XT_T3PMVQlk&export=download")
df4_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=167GBJwlhh58mVK-mSyJacQA8SB5uup93&export=download")
df5_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1w94ESSIUruinU_Pf--jkKk8B2-GDxRb6&export=download")
df6_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1N9MqB9DNpdGMWiK_fGXRf3FrosCjR9ob&export=download")
df7_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=16ipx0P4WY3SoVFN2LbVst1N_5yr98OYA&export=download")
df8_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1n8UpCl-qWXPvzrhmP7W9ISH9lyy-_K4V&export=download")
df_amazon   = pd.read_csv("https://drive.google.com/u/0/uc?id=1WAtbMW-d49_PPSQ6Kngq9NoD-dcOOiC9&export=download")
df_disney   = pd.read_csv("https://drive.google.com/u/0/uc?id=1Qn_Tv5vEmWg5PRqJkBqN_0SaSG0zFtuR&export=download")
df_hulu     = pd.read_csv("https://drive.google.com/u/0/uc?id=11FFXj7Dn22HjAnT3hk0XmTzav7jY9B0l&export=download")
df_netflix  = pd.read_csv("https://drive.google.com/u/0/uc?id=1cwzLeYjPIeJ5aJbyJO-F1QO5iHYIh6T2&export=download")

In [19]:
# concatenate the ratings dataframes into a single dataframe
df = pd.concat([df1_ratings, df2_ratings, df3_ratings, df4_ratings, 
                df5_ratings, df6_ratings, df7_ratings, df8_ratings])

Adjust decimals to round to 2

In [20]:
pd.set_option('display.float_format', '{:.2f}'.format)

Starting 2nd EDA

Check for nulls

In [21]:
df.isnull().sum()

userId       0
rating       0
timestamp    0
movieId      0
dtype: int64

In [27]:
df.head()

,userId,rating,timestamp,movieId
0,1,1.00,1425941529,as680
1,1,4.50,1425942435,ns2186
2,1,5.00,1425941523,hs2381
3,1,5.00,1425941546,ns3663
4,1,5.00,1425941556,as9500


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   rating     float64
 2   timestamp  int64  
 3   movieId    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 420.5+ MB


In [24]:
df.dtypes

userId         int64
rating       float64
timestamp      int64
movieId       object
dtype: object

In [25]:
df.describe()

,userId,rating,timestamp
count,11024289.00,11024289.00,11024289.00
mean,89972.51,3.53,1172541513.59
std,86866.01,1.06,205677385.96
min,1.00,0.50,789652004.00
25%,28558.00,3.00,990159330.00
50%,56843.00,3.50,1153610452.00
75%,116867.00,4.00,1360469934.00
max,270896.00,5.00,1501826675.00


Drop duplicates

In [26]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

Check for the unique values in some columns

In [28]:
np.sort(df['rating'].unique())

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

Drop column that is not going to be used

In [29]:
df = df.drop('timestamp', axis=1)

Second group of datasets

Create a column for the platform name

In [33]:
# Add platform column to each DataFrame
df_amazon['platform'] = 'amazon'
df_disney['platform'] = 'disney'
df_hulu['platform'] = 'hulu'
df_netflix['platform'] = 'netflix'

Concatenate the previous dataframes

In [34]:
# Concatenate the DataFrames into a single DataFrame
df_platforms = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], ignore_index=True)

Check for null values

In [42]:
df_platforms.isna().sum()

id               0
show_id          0
type             0
title            0
director      8260
cast          5321
country      11499
rating         864
duration       482
listed_in        0
platform         0
dtype: int64

Drop some columns that are not going to be used

In [39]:
df_platforms = df_platforms.drop(['date_added', 'release_year', 'description'], axis=1)

Create 'id' colum which uses the first character of the platform name plus the show id

In [37]:
df_platforms.insert(loc=0, column='id', value = df_platforms.apply(lambda x: x['platform'][0] + x['show_id'], axis=1))

Drop some more columns

In [45]:
df_platforms = df_platforms.drop('show_id', axis=1)

Converts the whole dataframe to lowercase

In [46]:
df_platforms.iloc[:] = df_platforms.iloc[:].applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [50]:
df_platforms.describe()

,duration_int
count,22998.00
mean,65.71
std,51.76
min,0.00
25%,3.00
50%,83.00
75%,101.00
max,601.00


Rename column

In [51]:
df = df.rename(columns={'movieId': 'id'})

Create a pandas series for the average rating for all movies

In [52]:
average = df.groupby('id')['rating'].mean()

Conver the previous series to a dataframe

In [53]:
df = average.reset_index()[['id', 'rating']]

In [55]:
df = df.rename(columns={'rating': 'score'})

Merge the new dataframe with the platforms dataframe

In [56]:
df = pd.merge(df_platforms, df, on='id')

Drop the remaining columns that are not going to be used

In [ ]:
df = df.drop(['id', 'platform', 'duration', 'rating', 'type', 'country', 'cast'], axis=1)

There are movie titles that appear in more than 1 row so I decide to delete the rows with the least amount of NaN values

In [63]:
# Group the rows by title and apply a function to select the row with the least NaN values
df = df.groupby('title', as_index=False).apply(lambda x: x.loc[x.isna().sum(axis=1).idxmin()])

# Drop any remaining duplicate rows
df.drop_duplicates(subset='title', inplace=True)

Sort the columns for better readability

In [65]:
df = df.reindex(columns=['title', 'score',  'director', 'listed_in'])

In [66]:
df = df.rename(columns={'listed_in': 'genre'})

In [73]:
#drop rows with missing values
df.dropna(inplace=True)

Export the clean data to be used for the recommendation system

In [75]:
df.to_csv('data_ML.csv', index=False)